 Set debug dot directory env variable.

In [ ]:
nb = "nb1"
dotdir = "/home/root/gst-dot/" + nb

# create directory for dot files
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

This step imports required python packages for this demo and initializes the GStreamer library.

In [ ]:
from IPython.display import Image, display, clear_output
import sys
import time
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

Gst.init(None);

Enable debug and set debug level.

In [ ]:
Gst.debug_set_active(False)
Gst.debug_set_default_threshold(3)

Create te file source element and set properties. Currently supported video codecs are theora, vp8 and vp9. Theora video files typically use an ogg container, whereas vp8 and vp9 video files typically use a matroska/webm container.

In [ ]:
file_name = "/usr/share/movies/Big_Buck_Bunny_4K.webm.360p.vp9.webm"
loop = True

src = Gst.ElementFactory.make("multifilesrc")
src.set_property("location", file_name)
src.set_property("loop", loop)

Create decodebin element. The decodebin instantiates the oggdemux and theoradec elements in this example. Create a videoconvert element that converts the I420 format to YUY2.

In [ ]:
decodebin = Gst.ElementFactory.make("decodebin")
cvt = Gst.ElementFactory.make("videoconvert")

Create jpeg encode element.

In [ ]:
jpegenc = Gst.ElementFactory.make("jpegenc")  

Create appsink element. Add callback function and connect to emit-signals.

In [ ]:
def new_sample(sink):  
    sample = sink.pull_sample()
    buffer = sample.get_buffer()
    ret, info = buffer.map(Gst.MapFlags.READ)
    
    display(Image(data=info.data))
    clear_output(wait=True)
    
    buffer.unmap(info)
    
    return Gst.FlowReturn.OK

In [ ]:
sink = Gst.ElementFactory.make("appsink")
sink.set_property("drop", True)
sink.set_property("max_buffers", 0)
sink.set_property("emit-signals", True)
sink.connect("new-sample", new_sample);

#print("sink properties: " + str(sink.get_property("drop")) + ", " + str(sink.get_property("max_buffers")))

The decoder source pad is created dynamically when it detects the incoming stream. The pad-added signal is emitted and this pad_added callback function is executed. It links the source pad of the decoder to the sink pad of the videoconvert element.

In [ ]:
def pad_added(element, pad):
    global cvt
    
    sink_pad = cvt.get_static_pad("sink")
    if not sink_pad.is_linked():
        pad.link(sink_pad)

Create pipeline, add and link elements. Note that the decodebin is not linked to the videoconvert element at this point. The link is created dynamically in the callback function once the decodbin pad is added.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(decodebin)
pipeline.add(cvt)
pipeline.add(jpegenc)
pipeline.add(sink)

src.link(decodebin)
cvt.link(jpegenc)
jpegenc.link(sink)

decodebin.connect("pad-added", pad_added);

This bus_call function listens on the bus for EOS and Error events.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline. Create main loop and listen to bus messages. Display the video frames in the notebook. 

To stop the pipeline, click the square shaped 'interrupt kernel' icon in the top menu bar. Creates a dot graph before stopping the pipeline.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)
try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

Show pipeline graph. This step may take a few seconds. Double click on the generate image file that shows the graph to zoom in.

In [ ]:
import pydot

# register dot plugins for png export to work
!dot -c

# convert dot file to png and display
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

The end.